<div style="overflow: hidden;">
    <img src="images/DREGS_logo_v2.png" width="300" style="float: left; margin-right: 10px;">
</div>

# Getting started: Part 3 - Simple queries

Here we continue our getting started tutorial, introducing queries.

### What we cover in this tutorial

In this tutorial we will learn how to:

1) Perform a simple query with a single filter
2) Perform a simple query with multiple filters
3) Query for all datasets tagged with a given keyword

### Before we begin

If you haven't done so already, check out the [getting setup](https://lsstdesc.org/dataregistry/tutorial_setup.html) page from the documentation if you want to run this tutorial interactively.

A quick way to check everything is set up correctly is to run the first cell below, which should load the `dataregistry` package, and print the package version.

In [ ]:
# Come up with a random owner name to avoid clashes
from random import randint
import os
OWNER = "tutorial_" + os.environ.get('USER') + '_' + str(randint(0,int(1e6)))

import dataregistry
print(f"Working with dataregistry version: {dataregistry.__version__} as random owner {OWNER}")

**Note** that running some of the cells below may fail, especially if run multiple times. This will likely be from clashes with the unique constraints within the database (hopefully the error output is informative). In these events either; (1) run the cell above to establish a new database connection with a new random user, or (2) manually change the conflicting database column(s) that are clashing during registration.

## 1) Querying the data registry with a single filter

Now that we have covered the basics of dataset registration, we can have a look at how to query entries in the database. Note you can only query for datasets within the schema you have connected to.

We learned how to connect to the DESC data registry in the last tutorial using the `DataRegistry` class, let's connect again using the defaults:

In [ ]:
from dataregistry import DataRegistry

# Establish connection to the tutorial schema
datareg = DataRegistry(schema="tutorial_working", owner=OWNER)

### Constructing the query 

Queries are constructed from one or more boolean logic "filters", which translate to SQL `WHERE` clauses in the code. 

For example, to create a filter that will query for all datasets in registry with the name "nersc_tutorial:my_first_desc_dataset" would be as follows:

In [ ]:
# Create a filter that queries on the dataset name
f = datareg.Query.gen_filter('dataset.name', '==', 'nersc_tutorial:my_first_desc_dataset')

Where the first argument is the column name we are searching against, the second argument is the logic operator, and the final argument is the condition. 

Like with SQL, column names can either be explicit, or not, with the prefix of their table name. For example `name` rather than `dataset.name`. However this would only be valid if the column `name` was unique across all tables in the database, which it is not. We would always recommend being explicit, and including the table name with filters.

The allowed boolean logic operators are: `==`, `!=`, `<`, `<=`, `>` and `>=`.

A special operator, `~=`, can be use to perform wildcard querties, where `*` is the wildcard character. This is particularly useful when only a partial dataset name is known, or when we want to return all datasets with a similar naming pattern, for example

In [ ]:
# Create a filter that queries on the dataset name with a wildcard
f = datareg.Query.gen_filter('dataset.name', '~=', 'nersc_tutorial:*')

will return all datasets whose name begins with the pattern `nersc_tutorial:`. The `~=` operator is case insensitive, for case sensitive wildcard searching, one can use the `~==` operator.

### Performing the query

Now we can pass this filter through to a query using the `Query` extension of the `DataRegistry` class, e.g.,

In [ ]:
# Query the database
results = datareg.Query.find_datasets(['dataset.dataset_id', 'dataset.name', 'dataset.relative_path'], [f])

Which takes a list of column names we want to return (in this case `dataset.dataset_id`, `dataset.name` and `dataset.relative_path`), and a list of filter objects for the query (just `f` in our case here).

We can look at the results like so:

In [ ]:
print(results)

### Query return formats

Note that three return formats are supported, selected via the optional `return_format` attribute passed to the `find_datasets` function:

- `return_format="property_dict"` : a dictionary with keys in the format `<table_name>.<column_name>` (default)
- `return_format="dataframe"` : a pandas DataFrame with keys in the format `<table_name>.<column_name>`
- `return_format="cursorresult"` : a SQLAlchemy CursorResult object (see [here](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.CursorResult) for details)

Note that for the `CursorResult` object, the property names are still in the format `<table_name>.<column_name>`. Because there is a `.` in the column names, to retrieve the properties you need to do `getattr(r, "dataset.name")`, where `r` is the row of the `CursorResult` object. 

To get a list of all columns in the database, along with what table they belong to, you can use the `Query.get_all_columns()` function, i.e.,

In [ ]:
print(datareg.Query.get_all_columns())

## 2) Querying the data registry with multiple filters

We are not limited to using a single filter during queries.

Now let's say we want to return all datasets in the registry with a particular `owner_type`, that were registered after a certain date. We also want the results in a Pandas dataframe format.

To do this we construct two filter objects, i.e.:

In [ ]:
# Create a filter that queries on the owner
f = datareg.Query.gen_filter('dataset.owner_type', '==', 'group')

# Create a 2nd filter that queries on the entry date
f2 = datareg.Query.gen_filter('dataset.creation_date', '>', '01-01-2024')

Then we query the database as before:

In [ ]:
# Query the database
results = datareg.Query.find_datasets(['dataset.dataset_id', 'dataset.name', 'dataset.owner',
                                       'dataset.relative_path', 'dataset.creation_date', 'dataset.owner_type'],
                                      [f,f2],
                                      return_format="dataframe")

and print the results

In [ ]:
print(results)

## 3) Query for all datasets with a keyword

We saw in the previous tutorial that we can tag datasets with one or more keywords.

To get back all datasets with a given keyword we construct a filter based on the `keyword` column from the `keyword` table. For example

In [ ]:
# Create a filter on a given keyword
f = datareg.Query.gen_filter('keyword.keyword', '==', 'simulation')

# Query the database for all datasets tagged with the keyword "simulation"
results = datareg.Query.find_datasets(['dataset.dataset_id', 'dataset.name', 'dataset.owner',
                                       'dataset.relative_path', 'dataset.creation_date'],
                                      [f],
                                      return_format="dataframe")

print(results)